#Классификация оттенка коротких текстов [SpaCy]

In [10]:
import spacy
import pandas as pd
from tqdm.auto import tqdm
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split


#Препроцессинг датасета

In [11]:
#Загрузка датасета
df = pd.read_csv('women-clothing-accessories.3-class.balanced.csv', encoding='utf8', sep='\t')
#Удаляем нейтральные комментарии
df = df[df['sentiment'] != 'neautral']
df

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


In [12]:
#Разделение на тренировочную и валидационную выборку
train_data, valid_data = train_test_split(df, test_size=0.3, random_state=42)

In [13]:
! python -m spacy download ru_core_news_sm

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB 1.4 MB/s eta 0:00:12
     ---------------------------------------- 0.0/15.3 MB 1.4 MB/s eta 0:00:12
     ---------------------------------------- 0.0/15.3 MB 1.4 MB/s eta 0:00:12
     ---------------------------------------- 0.0/15.3 MB 1.4 MB/s eta 0:00:12
     --------------------------------------- 0.1/15.3 MB 467.6 kB/s eta 0:00:33
     --------------------------------------- 0.1/15.3 MB 481.4 kB/s eta 0:00:32
      -------------------------------------- 0.2/15.3 MB 623.6 kB/s eta 0:00:25
      -------------------------------------- 0.3/15.3 MB 749.3 kB/s eta 0:00:21
      -------------------------------------- 0.3/15.3 MB 807.1 kB/s eta 0:00:19
      -------------------------------------- 0.4/15.3 MB 897.8 kB/s eta 0:00:17
      -------------------------------------- 0.4/15.3 MB 897.8 kB/s eta 0:00:17
     - ------------------------------------- 0.5/15.

In [14]:
nlp = spacy.load("ru_core_news_sm") # небольшой датасет на русском

In [15]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    # nlp.pipe([texts]) is way faster than running
    # nlp(text) for each text
    # as_tuples allows us to pass in a tuple,
    # the first one is treated as text
    # the second one will get returned as it is.
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        # One Hot Encodding
        if label == 'negative':
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        docs.append(doc)
    return docs

In [16]:
train_docs = make_docs(train_data.values)

  0%|          | 0/42000 [00:00<?, ?it/s]

In [17]:
valid_docs = make_docs(valid_data.values)

  0%|          | 0/18000 [00:00<?, ?it/s]

In [18]:
# Запись на диск
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")

doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")


#Работа с SpaCy

In [19]:
# на основе base_config сделать config.cfg
! python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [21]:
# тренируем модель
! python -m spacy train config.cfg --output ./output

ℹ Saving to output directory: output
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       60.94    0.61
  0     200         32.46       86.53    0.87
  0     400         21.90       88.54    0.89
  0     600         17.49       90.18    0.90
  0     800         13.82       90.64    0.91
  0    1000         13.99       91.28    0.91
  0    1200         14.52       91.80    0.92
  0    1400         12.79       92.19    0.92
  0    1600         12.41       92.12    0.92
  0    1800         11.80       92.48    0.92
  0    2000         11.66       92.89    0.93
✔ Saved pipeline to output directory
output\model-last


ℹ Saving to output directory: output
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       60.50    0.60
  0     200         36.14       85.63    0.86
  0     400         25.35       87.60    0.88
  0     600         19.35       89.21    0.89
  0     800         16.42       89.80    0.90
  0    1000         16.28       90.62    0.91
  0    1200         15.97       91.05    0.91
  0    1400         14.30       91.51    0.92
  0    1600         13.98       91.86    0.92
  0    1800         12.89       92.08    0.92
  0    2000         13.23       92.33    0.92
  0    2200         12.37       92.57    0.93
  0    2400         12.54

### Тест модели

In [ ]:
# загружаем лучшую модель
nlp = spacy.load("output/model-best")
texts = df[df['sentiment'] == 'positive'].head(2)['review'].values.tolist() + df[df['sentiment'] == 'negative'].head(2)['review'].values.tolist()
print(texts)
print("type : ‘quit’ to exit")
# predict the sentiment until someone writes quit
for text in texts:
    print(text)
    doc = nlp(text)
    print(doc.cats)
    if doc.cats['positive'] > .5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")


['Все совпадает! К покупке рекомендую!', 'по размеру) качественные)', 'качество плохое пошив ужасный (горловина наперекос) Фото не соответствует Ткань ужасная рисунок блеклый маленький рукав не такой УЖАС!!!!! не стоит за такие деньги г.......', 'Товар отдали другому человеку, я не получила посылку. Ладно хоть деньги вернули.']
type : ‘quit’ to exit
Все совпадает! К покупке рекомендую!
{'positive': 0.9371482729911804, 'negative': 0.06285169720649719}
the sentiment is positive
по размеру) качественные)
{'positive': 0.9235162138938904, 'negative': 0.07648380100727081}
the sentiment is positive
качество плохое пошив ужасный (горловина наперекос) Фото не соответствует Ткань ужасная рисунок блеклый маленький рукав не такой УЖАС!!!!! не стоит за такие деньги г.......
{'positive': 1.0939164894807618e-05, 'negative': 0.9999890327453613}
the sentiment is negative
Товар отдали другому человеку, я не получила посылку. Ладно хоть деньги вернули.
{'positive': 0.0497824028134346, 'negative': 0.95021